# Testing rpy2: Princals

We first start with setting up the environment and install the required R and Python packages:

In [1]:
!R -e "install.packages(c('lavaan', 'semTools', 'MPsychoR', 'lordif', 'Hmisc'), repos='https://cran.uni-muenster.de', quiet=TRUE)"
!pip install rpy2==3.5.17


R version 4.4.3 (2025-02-28) -- "Trophy Case"
Copyright (C) 2025 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> install.packages(c('lavaan', 'semTools', 'MPsychoR', 'lordif', 'Hmisc'), repos='https://cran.uni-muenster.de', quiet=TRUE)
also installing the dependencies ‘audio’, ‘globals’, ‘listenv’, ‘R.oo’, ‘R.methodsS3’, ‘zoo’, ‘permute’, ‘parallelly’, ‘beepr’, ‘RPushbullet’, ‘future’, ‘future.apply’, ‘progressr’, ‘R.utils’, ‘Mat

In [2]:
# General imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Rpy2 imports
from rpy2 import robjects as ro
from rpy2.robjects import pandas2ri, numpy2ri
from rpy2.robjects.packages import importr

# Automatic conversion of arrays and dataframes
pandas2ri.activate()
numpy2ri.activate()

# Set random seed for reproducibility
ro.r('set.seed(123)')

# Ipython extenrsion for magix plotting
%load_ext rpy2.ipython

# R imports
importr('base')
importr('lavaan')
importr('semTools')
importr('MPsychoR')
importr('lordif')
importr('Hmisc')




rpy2.robjects.packages.Package as a <module 'Hmisc'>

## 1. Measurement invariance across groups for quantitative item responses

Measurement invariance means that we measure the same psychological construct across groups and time (see lectures 8 & 9 for examples). In the following RMD we will look at how we can assess measurement invariance across...

* **groups** for **quantitative** item responses
* **groups** for **qualitative** item responses
* **time** for **quantitative** responses  

The four most important measurement invariance structures are the following:

* **Configural invariance:** same factor structure but unrestricted loadings and intercepts across groups.
* **Metric invariance:** the factor structure and the loadings are constrained to be equal across groups; the intercepts are free. (In the book this is referred to as *Weak invariance*)
* **Scale invariance:** the factor structure, the loadings and the indicator intercepts are restricted to be equal across groups. (In the book this is referred to as *Strong invariance*)
* **Strict invariance:** the factor structure, the loadings, the indicator intercepts and the reliabilities are restricted to be equal across groups.

Note. In the book there's is also a model in which, in addition to the indicator intercepts, the means of the latent
variables are restricted to be equal as well, however we won't cover this here.  

Try to load and inspect the dataset. It's called `Bergh`.

### Load, prepare and inspect the dataset

In [3]:
ro.r('data("Bergh")')
# Convert to Python
Bergh = pandas2ri.rpy2py(ro.globalenv['Bergh'])
Bergh.head()

,EP,SP,HP,DP,A1,A2,A3,O1,O2,O3,gender
1,2.666667,3.125,1.4,2.818182,3.4375,3.600000,3.352941,2.8750,3.400000,3.176471,1
2,2.666667,3.250,1.4,2.545455,2.3125,2.666667,3.117647,4.4375,3.866667,4.470588,1
3,1.000000,1.625,2.7,2.000000,3.5625,4.600000,3.941176,4.2500,3.666667,3.705882,1
4,2.666667,2.750,1.8,2.818182,2.7500,3.200000,3.352941,2.8750,3.400000,3.117647,1
5,2.888889,3.250,2.7,3.000000,3.2500,4.200000,3.764706,3.9375,4.400000,4.294118,1


#### The dataset

We illustrate todays topic using a dataset from Bergh et al. (2016). Part of their
analysis consisted of a multigroup CFA, where ethnic prejudice *(EP)*, sexism *(SP)*,
sexual prejudice against gays and lesbians *(HP)*, and prejudice toward mentally
people with disabilities (DP) were modeled *as indicators of* a generalized prejudice
factor *(GP)*. The multigroup aspect comes in by exploring differences in CFA parameters across *gender*.

### Fit the models

Luckily, we don't have to specify all the models mentioned above by ourselves. Instead, we can use the `measurementInvariance()` function from the `semTools` package. We only need to specify few elements:

1. )  The model equation (i.e. which items we want to include),
2. )  The dataset,
3. )  The grouping variable and
4. )  The estimator (We use a robust ML estimator since some of the prejudice measures deviate from normality).  

Note. `strict = T` gives us a configural, metric (weak), scale (strong), strict and a fifth model. With `strict = F` we won't get the strict model.

In [4]:
ro.r("GP_model <- 'GP =~ EP + HP + DP + SP'")
ro.r('minvfit <- measurementInvariance(model = GP_model, data = Bergh, group = "gender", estimator = "MLR", strict = T)')


Measurement invariance models:

Model 1 : fit.configural
Model 2 : fit.loadings
Model 3 : fit.intercepts
Model 4 : fit.residuals
Model 5 : fit.means


Scaled Chi-Squared Difference Test (method = “satorra.bentler.2001”)

lavaan->lavTestLRT():  
   lavaan NOTE: The “Chisq” column contains standard test statistics, not the 
   robust test that should be reported per model. A robust difference test is 
   a function of two standard (not robust) statistics.
               Df    AIC    BIC    Chisq Chisq diff Df diff Pr(>Chisq)    
fit.configural  4 7376.1 7490.2   1.6552                                  
fit.loadings    7 7382.9 7482.8  14.4960     11.834       3   0.007972 ** 
fit.intercepts 10 7417.7 7503.3  55.2875     47.833       3  2.311e-10 ***
fit.residuals  14 7429.8 7496.4  75.3689     12.468       4   0.014189 *  
fit.means      15 7482.4 7544.3 130.0375     49.267       1  2.234e-12 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Fit measures:

       

This code chunk fits four invariance models: configural, metric (weak), scale (strong), strict and a fifth model where, in addition to the indicator intercepts, the means of the latent variables are restricted to be equal as well. As mentioned above, we won't deal with the last model.  

Each model is tested against the previous one, and a **significant result** indicates that the higher restricted model is significantly **worse** than the previous model. In our example, only configural variance holds as restricting the model parameters to match the assumptions of metric invariance results in a significantly worse fit.

#### Select the best fitting model

In our example we would go with the configural model, for which the output can be requested as follows:

In [5]:
# Extract only the configural model
ro.r("Fit_conf <- minvfit$fit.configural")

Try to investigate the `Fit_conf` using `summary()`. Remember to set `standardized = TRUE` & `fit.measures = TRUE`.

In [6]:
summary_conf = ro.r("summary(Fit_conf, standardized = TRUE, fit.measures = TRUE)")
print(summary_conf)

lavaan 0.6-19 ended normally after 45 iterations

  Estimator                                         ML
  Optimization method                           NLMINB
  Number of model parameters                        24

  Number of observations per group:                   
    male                                           249
    female                                         612

Model Test User Model:
                                              Standard      Scaled
  Test Statistic                                 1.655       1.549
  Degrees of freedom                                 4           4
  P-value (Chi-square)                           0.799       0.818
  Scaling correction factor                                  1.068
    Yuan-Bentler correction (Mplus variant)                       
  Test statistic for each group:
    male                                         0.348       0.348
    female                                       1.201       1.201

Model Test Baseline Model

For the interpretation of this output, please refer to the previous sessions.

### Another Option

Instead of using these generic invariance sequences of models, we can approach
multigroup CFA from a more hypothesis-driven direction. In the first model we are
going to fit, the loadings for ethnic minorities (EP), gays/lesbians (HP), and people
with disabilities (DP) are held equal across men and women. We keep the sexism
**(SP) loadings free**. Note that all intercepts are free to vary.  

In `lavaan`, this can be achieved by using one of the following **two equivalent model formulations**.
In the first variant, we explicitly incorporate the loadings
restrictions in the model formulation. We specify a vector of length 2, the first
element denoting the loading for the first gender category and the second element
denoting the loading for the second category. Since we use the same loadings
symbols for both categories, they are restricted to be equal.

#### Option 1 to specify the model

In [7]:
# First option to specify the model # CHECK: The values align with R but the readability not so much...
ro.r ("GP_model <-'GP =~ c(v1,v1)*EP + c(v2,v2)*HP + c(v3,v3)*DP + SP'") # The loading of SP is not fixed.
ro.r('fitBase <-lavaan::cfa(GP_model, data = Bergh, group = "gender", estimator = "MLR")')
ro.r("fitMeasures(fitBase)")

array([ 2.20000000e+01,  4.25165558e-03,  7.32135091e+00,  6.00000000e+00,
        2.92148336e-01,  6.85411314e+00,  6.00000000e+00,  3.34549818e-01,
        1.06816896e+00,  7.16580288e+02,  1.20000000e+01,  0.00000000e+00,
        5.30342427e+02,  1.20000000e+01,  0.00000000e+00,  1.35116530e+00,
        9.98124627e-01,  9.96249254e-01,  9.98352222e-01,  9.96704444e-01,
        9.98697343e-01,  9.97394686e-01,  9.96249254e-01,  9.79565860e-01,
        9.89782930e-01,  4.94891465e-01,  9.98140462e-01,  9.98124627e-01,
        9.96704444e-01,  9.74152122e-01,  9.87076061e-01,  4.93538031e-01,
        9.98371078e-01,  9.98352222e-01,  9.97394686e-01,  9.98697343e-01,
       -3.66686045e+03, -3.66319978e+03,  7.37772090e+03,  7.48239898e+03,
        8.61000000e+02,  7.41253284e+03,  1.25325348e+00,  1.19508682e+00,
        2.26176188e-02,  0.00000000e+00,  6.94429738e-02,  9.00000000e-01,
        7.89925895e-01,  5.00000000e-02,  1.69311900e-02,  8.00000000e-02,
        1.81842544e-02,  

#### Option 2 to specify the model

In the second variant, we restrict all the loadings to be equal across groups
using the `group.equal` argument. We then free up the SP loading through the
`group.partial` argument.


In [8]:
# Second option to specify the model
ro.r("GP_model <- 'GP =~ EP + HP + DP + SP'")
ro.r('fitBase <- lavaan::cfa(GP_model,data = Bergh, group = "gender", group.equal = c("loadings"), group.partial = c("GP=~ SP"), estimator = "MLR")')
ro.r("fitMeasures(fitBase)")

array([ 2.20000000e+01,  4.25165558e-03,  7.32135091e+00,  6.00000000e+00,
        2.92148336e-01,  6.85411314e+00,  6.00000000e+00,  3.34549818e-01,
        1.06816896e+00,  7.16580288e+02,  1.20000000e+01,  0.00000000e+00,
        5.30342427e+02,  1.20000000e+01,  0.00000000e+00,  1.35116530e+00,
        9.98124627e-01,  9.96249254e-01,  9.98352222e-01,  9.96704444e-01,
        9.98697343e-01,  9.97394686e-01,  9.96249254e-01,  9.79565860e-01,
        9.89782930e-01,  4.94891465e-01,  9.98140462e-01,  9.98124627e-01,
        9.96704444e-01,  9.74152122e-01,  9.87076061e-01,  4.93538031e-01,
        9.98371078e-01,  9.98352222e-01,  9.97394686e-01,  9.98697343e-01,
       -3.66686045e+03, -3.66319978e+03,  7.37772090e+03,  7.48239898e+03,
        8.61000000e+02,  7.41253284e+03,  1.25325348e+00,  1.19508682e+00,
        2.26176188e-02,  0.00000000e+00,  6.94429738e-02,  9.00000000e-01,
        7.89925895e-01,  5.00000000e-02,  1.69311900e-02,  8.00000000e-02,
        1.81842544e-02,  

Both variants lead to the same results. Using the `fitMeasures(fitBase)`
call, we get a $\chi^2$-value of 7.321 (df = 6, p = 0.292), a RMSEA of 0.023 with a 90%
CI of [0, 0.069], a CFI of 0.998, and an SRMR of 0.028. The model fits well.  

Since the difference in intercepts between men and women is negligible for the
EP indicator, we can force the intercepts to be equal for this indicator. Let us build on
the second specification variant from above, in order to set up this model. Through
group.equal we **constrain all loadings and intercepts to be equal across groups**
and, subsequently, **free up the SP loading as well as the intercepts for DP, HP, and**
**SP** using `group.partial`:

In [9]:
ro.r("GP_model <- 'GP =~ EP + HP + DP + SP'")

ro.r('fitBase1 <- lavaan::cfa(GP_model, data = Bergh,\n'
      'group = "gender", group.equal = c("loadings", "intercepts"),\n'
      'group.partial = c("GP=~SP", "DP~1", "HP~1", "SP~1"),\n'
      'estimator = "MLR")')

ro.r("fitMeasures(fitBase1)")


array([ 2.20000000e+01,  4.25165558e-03,  7.32135091e+00,  6.00000000e+00,
        2.92148336e-01,  6.85411326e+00,  6.00000000e+00,  3.34549806e-01,
        1.06816894e+00,  7.16580288e+02,  1.20000000e+01,  0.00000000e+00,
        5.30342427e+02,  1.20000000e+01,  0.00000000e+00,  1.35116530e+00,
        9.98124627e-01,  9.96249254e-01,  9.98352222e-01,  9.96704444e-01,
        9.98697343e-01,  9.97394685e-01,  9.96249254e-01,  9.79565860e-01,
        9.89782930e-01,  4.94891465e-01,  9.98140462e-01,  9.98124627e-01,
        9.96704444e-01,  9.74152122e-01,  9.87076061e-01,  4.93538030e-01,
        9.98371077e-01,  9.98352222e-01,  9.97394685e-01,  9.98697343e-01,
       -3.66686045e+03, -3.66319978e+03,  7.37772090e+03,  7.48239898e+03,
        8.61000000e+02,  7.41253284e+03,  1.25325348e+00,  1.14728335e+00,
        2.26176188e-02,  0.00000000e+00,  6.94429738e-02,  9.00000000e-01,
        7.89925895e-01,  5.00000000e-02,  1.69311900e-02,  8.00000000e-02,
        1.81842557e-02,  

Again, this model fits well. We now use this model as **baseline model** and
**compare it to two additional models**. First **(1)**, we **constrain the SP loading to be 0**
for the **women** (ingroup-outgroup model).We can specify this restriction directly in
the model specification through `c(NA,0)`. *NA means free to vary* across men (first
group), whereas the second element fixes the parameter to 0 for the women. Note
that this is quite a restrictive assumption.

In [10]:
ro.r("GP_model2 <- 'GP =~ c(v1,v1)*EP + c(v2,v2)*HP + c(v3,v3)*DP + c(NA, 0)*SP'")

ro.r('fitIO <- lavaan::cfa(GP_model2, data = Bergh,\n'
      'group = "gender", group.equal = c("intercepts"),\n'
      'group.partial = c("DP~1", "HP~1", "SP~1"),\n'
      'estimator = "MLR")')

ro.r("fitMeasures(fitIO)")


array([ 2.10000000e+01,  1.35746530e-01,  2.33755525e+02,  7.00000000e+00,
        0.00000000e+00,  1.98735302e+02,  7.00000000e+00,  0.00000000e+00,
        1.17621541e+00,  7.16580288e+02,  1.20000000e+01,  0.00000000e+00,
        5.30342427e+02,  1.20000000e+01,  0.00000000e+00,  1.35116530e+00,
        6.78169360e-01,  4.48290332e-01,  6.30099155e-01,  3.65884267e-01,
        6.77994193e-01,  4.47990045e-01,  4.48290332e-01,  4.40783170e-01,
        6.73790183e-01,  3.93044273e-01,  6.80437114e-01,  6.78169360e-01,
        3.65884267e-01,  3.57605443e-01,  6.25269842e-01,  3.64740741e-01,
        6.33633179e-01,  6.30099155e-01,  4.47990045e-01,  6.77994193e-01,
       -3.78007754e+03, -3.66319978e+03,  7.60215508e+03,  7.70207506e+03,
        8.61000000e+02,  7.63538466e+03,  1.25325348e+00,  1.11906623e+00,
        2.74311074e-01,  2.44736269e-01,  3.05059830e-01,  9.00000000e-01,
        0.00000000e+00,  5.00000000e-02,  1.00000000e+00,  8.00000000e-02,
        2.52240862e-01,  

It gives a $\chi^2$-value of 233.756 (df = 7, p = 0), a RMSEA of 0.274 with a 90%
CI of [0.245, 0.305], a CFI of 0.678, and an SRMR of 0.148. Bad fit.  

In the next model **(2)**, we **restrict all the loadings to be equal** (**marginalization**
**model**). The intercepts have the same constraints as above.

In [11]:
ro.r('fitMarg <- lavaan::cfa(GP_model, data = Bergh,\n'
      'group = "gender", group.equal = c("loadings", "intercepts"),\n'
      'group.partial = c("DP~1", "HP~1", "SP~1"),\n'
      'estimator = "MLR")')

ro.r("fitMeasures(fitMarg)")


array([ 2.10000000e+01,  8.41811186e-03,  1.44959886e+01,  7.00000000e+00,
        4.30310073e-02,  1.34775671e+01,  7.00000000e+00,  6.12928392e-02,
        1.07556419e+00,  7.16580288e+02,  1.20000000e+01,  0.00000000e+00,
        5.30342427e+02,  1.20000000e+01,  0.00000000e+00,  1.35116530e+00,
        9.89361058e-01,  9.81761814e-01,  9.87503305e-01,  9.78577095e-01,
        9.90052292e-01,  9.82946787e-01,  9.81761814e-01,  9.65321030e-01,
        9.79770601e-01,  5.71532851e-01,  9.89436025e-01,  9.89361058e-01,
        9.78577095e-01,  9.56434938e-01,  9.74587047e-01,  5.68509111e-01,
        9.87622698e-01,  9.87503305e-01,  9.82946787e-01,  9.90052292e-01,
       -3.67044777e+03, -3.66319978e+03,  7.38289554e+03,  7.48281553e+03,
        8.61000000e+02,  7.41612512e+03,  1.25325348e+00,  1.10248592e+00,
        4.98745352e-02,  8.45147096e-03,  8.63560095e-02,  9.00000000e-01,
        4.48041192e-01,  5.00000000e-02,  9.26516415e-02,  8.00000000e-02,
        4.63628755e-02,  

We get a $\chi^2$-value of 14.496 (df = 7, p = 0.043), a RMSEA of 0.05 with a 90%
CI of [0.008, 0.086], a CFI of 0.989, and an SRMR of 0.036. In terms of goodness-of-fit statistics, we do not see much of a difference compared to the baseline model.
Let us do some model comparison. Since the marginalization model is nested in
the basemodel, we can apply the following $\chi^2$-difference test. Note that a **significant**
**result suggests that the higher parametrized model fits significantly worse**.

In [12]:
print(ro.r("anova(fitMarg, fitBase1)"))


Scaled Chi-Squared Difference Test (method = “satorra.bentler.2001”)

lavaan->lavTestLRT():  
   lavaan NOTE: The “Chisq” column contains standard test statistics, not the 
   robust test that should be reported per model. A robust difference test is 
   a function of two standard (not robust) statistics.
         Df    AIC    BIC   Chisq Chisq diff Df diff Pr(>Chisq)  
fitBase1  6 7377.7 7482.4  7.3214                                
fitMarg   7 7382.9 7482.8 14.4960     6.4063       1    0.01137 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1



Even though significant, the AIC/BIC barely differ across the two groups,
and from the fit indices above, we learned that both models fit well. Thus, the
**marginalization model may be an attractive option to go with**.  
    
#### Didn't get it? No worries!

The alternative approach presented here may be quite confusing as it wasn't covered in the lecture. So lets put it in words.  

The goal here is to investigate the between-group measurement (in)variance of *one specific* indicator. In the approach mentioned before (and in the lecture) we investigated whether measurement invariance holds for *all* indicators.  

As you might have guessed, the indicator we are investigating in this alternative approach is `SP`. Lets revisit the steps:

1. We fit a model in which all loadings but the one from `SP` are fixed (`FitBase1`).

2. We fit alternative models with alternative properties of the `SP` laodings (all the other loadings stay the same). (`GP_model2`: Fix `SP` loading for women but not for men; `fitMarg`: Fix all loadings).

3. We compare model fits. Lets check the comparison of `fitMarg`and `fitBase1` again:

In [13]:
print(ro.r("anova(fitMarg, fitBase1)"))


Scaled Chi-Squared Difference Test (method = “satorra.bentler.2001”)

lavaan->lavTestLRT():  
   lavaan NOTE: The “Chisq” column contains standard test statistics, not the 
   robust test that should be reported per model. A robust difference test is 
   a function of two standard (not robust) statistics.
         Df    AIC    BIC   Chisq Chisq diff Df diff Pr(>Chisq)  
fitBase1  6 7377.7 7482.4  7.3214                                
fitMarg   7 7382.9 7482.8 14.4960     6.4063       1    0.01137 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1



Before we look at the values, lets think about what this comparison is good for. In the `fitBase1` model we allow the loadings of `SP` to vary between men and women. In the `fitMarg` model we assume them to be equal for both groups. Now we can apply our 'normal' principle again: If the more restrictive model that assumes measurement invariance (`fitMarg`) **is not significantly worse** than the model that allows the loadings to vary (`fitBase1`) **we assume measurement invariance**. The critical point here is that we only investigated measurement invaraince for *one* indicator (`SP`) and not for the whole model (as done in the previous section).

## 2. Measurement invariance across groups for qualitative item responses

Measurement invariance can also be tested for dichotomous or polytomous (i.e. qualitative) item responses. In the Item Response Theory (IRT) this is known as **Differential Item Functioning (DIF)**. We can distinguish between two forms of DIF:  

* **Uniform DIF:** the ICCs are shifted in location across subgroups, but they remain
parallel (i.e., a group main effect)
* **Nonuniform DIF:** the ICCs across subgroups are shifted, and they cross (i.e., an
interaction effect between group and the trait)  

We focus on DIF detection using logistic regression (we skip tree-based DIF assessment as it wasn't covered in the statistics lecture yet).  

The idea of this approach (Zumbo, 1999) is to specify a set of logistic regression equations and predict the original item responses from the person parameters $\theta$ and the external grouping variable $z$. The following set of proportional odds models is formulated (Choi et al., 2011):  

* $M_1$ : $logit(P(x_i)) = \tau_0 + \tau_1\theta$
* $M_2$ : $logit(P(x_i)) = \tau_0 + \tau_1\theta + \tau_2z$
* $M_3$ : $logit(P(x_i)) = \tau_0 + \tau_1\theta + \tau_2z + \tau_3\theta z$

We are interested in comparing the following models via the **LR-principle**:

* $M_2$ vs. $M_1$: if significant, we have **uniform** DIF.
* $M_3$ vs. $M_2$: if significant, we have **nonuniform** DIF.

### Load, prepare and inspect the dataset

Try to load and inspect the dataset. It's called `YouthDep`. Extract the first 26 items and store the result in `cdi`. Also inspect your newly created subset.

In [14]:
ro.r('data("YouthDep")')
# Convert to Python
YouthDep_df = pandas2ri.rpy2py(ro.globalenv['YouthDep'])
# Select the first 26 columns
cdi = YouthDep_df.iloc[:, :26]
print(cdi.head())

# Put data back into R
ro.globalenv['cdi'] = cdi

   CDI1  CDI2r  CDI3  CDI4  CDI5r  CDI6  CDI7r  CDI8r  CDI10r  CDI11r  ...  \
1     1      1     1     1      1     1      1      1       1       1  ...   
2     1      1     1     1      1     1      1      1       1       1  ...   
3     1      1     1     1      1     2      1      1       1       1  ...   
4     1      1     1     1      1     1      1      2       1       1  ...   
5     1      1     1     2      1     2      2      1       1       2  ...   

   CDI18r  CDI19  CDI20  CDI21r  CDI22  CDI23  CDI24r  CDI25r  CDI26  CDI27  
1       1      1      2       2      1      1       1       1      2      1  
2       1      1      1       1      1      1       1       1      1      1  
3       1      1      1       1      1      1       1       1      1      1  
4       1      1      1       1      1      2       2       1      1      1  
5       1      1      1       1      2      1       3       1      1      1  

[5 rows x 26 columns]


#### The dataset

As an example, we use a dataset from Vaughn-Coaxum et al. (2016) on youth
depression. The 26 items come from the Children’s Depression Inventory (CDI);
each item is scored on a scale from 0 to 2. The authors were interested in DIF
analyses on an external race variable (four categories). Note that the aim was
not to eliminate items from the CDI, which is a well-established scale. Rather,
the authors wanted to identify DIF items (which already gives useful substantive
information) and score all individuals in a “fair” way by means of group-specific
person parameter estimates for items flagged as DIF.

### Fit the models

Use the `lordif` function to fit the models (with this one function you will fit all the models described above). We want to use `YouthDep$race` as our grouping variable and our `cdi` subset as the dataset.

In [15]:
ro.r('cdiDIF <- lordif(cdi, YouthDep$race, criterion = "Chisqr")')


 (mirt) | Iteration: 1, 21 items flagged for DIF (1,2,4,5,6,7,9,10,11,13,14,15,16,17,18,20,22,23,24,25,26)
 (mirt) | Iteration: 2, 20 items flagged for DIF (2,3,4,5,6,7,8,10,11,13,14,16,17,18,20,22,23,24,25,26)
 (mirt) | Iteration: 3, 20 items flagged for DIF (2,3,4,5,6,7,8,10,11,13,14,16,17,18,20,22,23,24,25,26)


In total, 20 out of 26 items are flagged as DIF. Let us print out the p-values of
the LR-tests for the first three items:

In [16]:
print(ro.r("cdiDIF$stats[1:3, 1:5]"))

  item ncat chi12  chi13  chi23
1    1    2 0.352 0.3799 0.3718
2    2    2 0.000 0.0000 0.2084
3    3    2 0.000 0.0000 0.0022



We see that for the first item, none of the LR-$\chi$2 values is significant. In fact, item
1 was not flagged. For the second item, $\chi^2_{12}$ (i.e., M2 vs. M1) is significant, whereas $\chi^2_{23}$
(i.e., M3 vs. M2) is not significant. Thus, the second item has uniform DIF. For
the third item, all p-values are significant; we have the case of nonuniform DIF.
Corresponding plots can be produced as follows:  

**Note. The `plot()` function does not work in RMD with `lordif`objects.** If you want to see the plots, paste the following code chunk in a new R script (not an RMD!). You will end up with an empty plot in your plot window, however you can use the arrow button on the top left of the code window to cycle through the plots for every item. If you want to include the plots in your portfolio RMD follow this work-around: (1) Run the code chunk in a separate R script (adapted to your dataset) (2) Cycle through the plot using the arrow buttons in the plot window (3) Export the plot as a .png using the Export button in the plot window (4) insert the .png file in you RMD.

In [ ]:
# CHECK: What do we do about that? See above...

Let us print out the GRM parameters (discrimination, category boundaries) for
the six non-DIF items (non-DIF) and the first DIF item (I2):

In [17]:
GRM = ro.r("cdiDIF$ipar.sparse")
print(GRM.head(10))

            a       cb1      cb2
I1   2.055572 1.5197578       NA
I9   1.942808 1.6169260       NA
I12  1.224337 0.3175138 2.674436
I15  1.424712 1.1699027 2.507799
I19  2.107288 1.1153563       NA
I21  1.129338 1.9135221       NA
I2.1 1.638530 0.9251430       NA
I2.2 1.564181 0.9395047       NA
I2.3 1.611274 0.5323336       NA
I2.4 1.987255 1.0579461       NA



Note that for some items, there is only one category boundary. This results from
the fact that there were not enough observations in a particular category (here,
category 2) for parameter estimation. For such cases, `lordif` collapses categories
automatically. Item 2 was flagged as DIF.We get four sets of discrimination/boundary
parameters, one of each race category.  

The calibrated, group-specific person parameter vector can be extracted using:

In [18]:
ro.r("ppar <- cdiDIF$calib.sparse$theta")

Based on the DIF subgroup structure, they are fairly scored, are on the same
scale, and can be subject to further analysis.

## 3. Measurement invariance across time for quantitative item responses

CFA can also be extended to longitudinal settings, where indicators are presented
to the same individual at multiple points in time $t$. Across the time points,
we can consider the same measurement invariance principles as in multigroup CFA
(configural, weak, and strong invariance). However, in longitudinal CFA we need
to account for **correlated residuals** (i.e. autoregressive effects) since we cannot assume that independence holds
across time points.

### Load, prepare and inspect the dataset

In [19]:
ro.r('data("SDOwave")')
# Convert to Python
SDOwave = pandas2ri.rpy2py(ro.globalenv['SDOwave'])
print(SDOwave.head())

   I1.1996  I2.1996  I3.1996  I4.1996  I1.1997  I2.1997  I3.1997  I4.1997  \
1      2.0      2.0      4.0      4.0      1.0      1.0      4.0      1.0   
2      1.0      1.0      2.0      2.0      1.0      1.0      2.0      3.0   
3      1.0      1.0      1.0      1.0      1.0      1.0      1.0      1.0   
4      2.0      2.0      3.0      3.0      2.0      2.0      2.0      2.0   
5      1.0      1.0      6.0      4.0      7.0      1.0      1.0      1.0   

   I1.1998  I2.1998  I3.1998  I4.1998  I1.1999  I2.1999  I3.1999  I4.1999  \
1      2.0      1.0      4.0      2.0      2.0      3.0      3.0      1.0   
2      4.0      2.0      5.0      4.0      3.0      2.0      4.0      4.0   
3      1.0      1.0      2.0      1.0      1.0      1.0      6.0      1.0   
4      2.0      2.0      2.0      2.0      2.0      1.0      1.0      1.0   
5      1.0      1.0      4.0      1.0      4.0      1.0      3.0      2.0   

   I1.2000  I2.2000  I3.2000  I4.2000  
1      2.0      1.0      5.0      

#### The dataset

To illustrate longitudinal CFA, we use a dataset on social dominance orientation
(SDO; Sidanius and Pratto, 2001). SDO is assessed on the same individuals from
1996 to 2000 (Sidanius et al., 2010), involving the following four items, scored on a 7-point scale:  

* It is probably a good thing that certain groups are at the top and other groups are at the bottom (I1).

* Inferior groups should stay in their place (I2).

* We should do what we can to equalize conditions for different groups (I3, reversed).

* Increased social equality is beneficial to society (I4, reversed).

For the moment, let us consider a simple latent variable structure where all four
items load on a general SDO dimension. We pick 2 years only: 1996 vs. 1998.  

The most relevant model within this context is the **strong/scale invariance model**, which allows us to compare the latent means across the two time points. We restrict the factor loadings as well as the corresponding intercepts of each item to be equal in both measurement occasions (**strong/scale invariance**). Also, we need to allow for residual covariances (using the `~~` symbol in the syntax) for each item across time points. With this we account for autoregressive effects, which has to be done when modeling longitudinal data.

### Fit the models

We'll start with the scale invariance model and work our way down to lower levels of measurement invariance. (We exclude the strict invariance model here).

We'll start with the scale invariance model and work our way down to lower levels of measurement invariance. (We exclude the strict invariance model here).

#### Scale Invariance

In [20]:
# Specify the model
ro.r('model_scale <- "'
      'SDO1996 =~ 1*I1.1996 + a2*I2.1996 + a3*I3.1996 + a4*I4.1996\n'
      'SDO1998 =~ 1*I1.1998 + a2*I2.1998 + a3*I3.1998 + a4*I4.1998\n'
      'SDO1996 ~~ SDO1998\n'
      'I1.1996 ~ int1*1; I1.1998 ~ int1*1\n'
      'I2.1996 ~ int2*1; I2.1998 ~ int2*1\n'
      'I3.1996 ~ int3*1; I3.1998 ~ int3*1\n'
      'I4.1996 ~ int4*1; I4.1998 ~ int4*1\n'
      'I1.1996 ~~ I1.1998\n'
      'I2.1996 ~~ I2.1998\n'
      'I3.1996 ~~ I3.1998\n'
      'I4.1996 ~~ I4.1998\n'
      'SDO1996 ~ 0*1\n'
      'SDO1998 ~ 1"')

# Fit the model
ro.r('fit_scale <- lavaan::cfa(model_scale, data = SDOwave, estimator = "MLR")')

# Print the model output with fit measures and standardized estimates
summary_fit_scale = ro.r('summary(fit_scale, fit.measures=TRUE, standardized=TRUE)')
print(summary_fit_scale)

lavaan 0.6-19 ended normally after 45 iterations

  Estimator                                         ML
  Optimization method                           NLMINB
  Number of model parameters                        30
  Number of equality constraints                     7

  Number of observations                           612

Model Test User Model:
                                              Standard      Scaled
  Test Statistic                               315.748     203.519
  Degrees of freedom                                21          21
  P-value (Chi-square)                           0.000       0.000
  Scaling correction factor                                  1.551
    Yuan-Bentler correction (Mplus variant)                       

Model Test Baseline Model:

  Test statistic                              1993.536    1098.021
  Degrees of freedom                                28          28
  P-value                                        0.000       0.000
  Scaling correcti

The parameters of main interest are the latent variable means (however, this has nothing to do with measurement invariance but can be used to compare latent score bewteen time points):

In [26]:
print(ro.r("parameterEstimates(fit_scale)[22:23,]"))

       lhs op rhs label    est    se      z pvalue ci.lower ci.upper
22 SDO1996 ~1            0.000 0.000     NA     NA    0.000    0.000
23 SDO1998 ~1           -0.047 0.019 -2.432  0.015   -0.085   -0.009



We see that the mean for 1996 was fixed to 0 and is therefore used as the reference
year. The second line suggests that there is a significant decrease in SDO from 1996
to 1998.  

Note that a **weak/metric invariance** version of this model can be fitted by *restricting both*
*latent means to 0* and *freeing up the intercepts*. For a **configural invariance** version,
the *loadings need to be freed up as well* (first two syntax lines). For a **strict invariance** version, *intercepts, loadings and residuals* need to be *fixed*.  
If we do not want to do this by hand, the `longInvariance` function from `semTools` can be used
to establish such a generic sequence (similar to the `measurementInvariance` function used above).
Using the `anova` function, the models can be again tested against each other.  

Lets fit a **weak/metric invariance** model.

#### Metric Invariance

Adapt the code from the scale model to fit a metric model by freeing up the intercepts and restrict both latent variables to zero. Name your model `model.metric`.

In [27]:
# Specify the model
ro.r('model_metric <- "'
      'SDO1996 =~ 1*I1.1996 + a2*I2.1996 + a3*I3.1996 + a4*I4.1996\n'
      'SDO1998 =~ 1*I1.1998 + a2*I2.1998 + a3*I3.1998 + a4*I4.1998\n'
      'SDO1996 ~~ SDO1998\n'
      'I1.1996 ~ 1; I1.1998 ~ 1\n'
      'I2.1996 ~ 1; I2.1998 ~ 1\n'
      'I3.1996 ~ 1; I3.1998 ~ 1\n'
      'I4.1996 ~ 1; I4.1998 ~ 1\n'
      'I1.1996 ~~ I1.1998\n'
      'I2.1996 ~~ I2.1998\n'
      'I3.1996 ~~ I3.1998\n'
      'I4.1996 ~~ I4.1998\n'
      'SDO1996 ~ 0*1\n'
      'SDO1998 ~ 0*1"')

# Fit the model
ro.r('fit_metric <- lavaan::cfa(model_metric, data = SDOwave, estimator = "MLR")')

# Print the model output with fit measures and standardized estimates
summary_fit_metric = ro.r('summary(fit_metric, fit.measures=TRUE, standardized=TRUE)')
print(summary_fit_metric)

lavaan 0.6-19 ended normally after 43 iterations

  Estimator                                         ML
  Optimization method                           NLMINB
  Number of model parameters                        29
  Number of equality constraints                     3

  Number of observations                           612

Model Test User Model:
                                              Standard      Scaled
  Test Statistic                               310.762     189.360
  Degrees of freedom                                18          18
  P-value (Chi-square)                           0.000       0.000
  Scaling correction factor                                  1.641
    Yuan-Bentler correction (Mplus variant)                       

Model Test Baseline Model:

  Test statistic                              1993.536    1098.021
  Degrees of freedom                                28          28
  P-value                                        0.000       0.000
  Scaling correcti

Note that the values in the `intercepts` section are now varying.

Again, we can extract latent variable means (however, this has again nothing to do with measurement invariance):

In [28]:
print(ro.r("parameterEstimates(fit_metric)[22:23,]"))

       lhs op rhs label est se  z pvalue ci.lower ci.upper
22 SDO1996 ~1             0  0 NA     NA        0        0
23 SDO1998 ~1             0  0 NA     NA        0        0



Lets continue with a **configural variance** model.

#### Configural Invariance

Adapt the code from the metric model to fit a configural model by freeing up the loadings as well (first two syntax lines). Name your model `model.configural`.

In [29]:
# Specify the model
ro.r('model_configural <- "'
      'SDO1996 =~ 1*I1.1996 + I2.1996 + I3.1996 + I4.1996\n'
      'SDO1998 =~ 1*I1.1998 + I2.1998 + I3.1998 + I4.1998\n'
      'SDO1996 ~~ SDO1998\n'
      'I1.1996 ~ 1; I1.1998 ~ 1\n'
      'I2.1996 ~ 1; I2.1998 ~ 1\n'
      'I3.1996 ~ 1; I3.1998 ~ 1\n'
      'I4.1996 ~ 1; I4.1998 ~ 1\n'
      'I1.1996 ~~ I1.1998\n'
      'I2.1996 ~~ I2.1998\n'
      'I3.1996 ~~ I3.1998\n'
      'I4.1996 ~~ I4.1998\n'
      'SDO1996 ~ 0*1\n'
      'SDO1998 ~ 0*1"')

# Fit the model
ro.r('fit_configural <- lavaan::cfa(model_configural, data = SDOwave, estimator = "MLR")')

# Print the model output with fit measures and standardized estimates
summary_fit_configural = ro.r('summary(fit_configural, fit.measures=TRUE, standardized=TRUE)')
print(summary_fit_configural)

lavaan 0.6-19 ended normally after 61 iterations

  Estimator                                         ML
  Optimization method                           NLMINB
  Number of model parameters                        29

  Number of observations                           612

Model Test User Model:
                                              Standard      Scaled
  Test Statistic                               301.687     175.637
  Degrees of freedom                                15          15
  P-value (Chi-square)                           0.000       0.000
  Scaling correction factor                                  1.718
    Yuan-Bentler correction (Mplus variant)                       

Model Test Baseline Model:

  Test statistic                              1993.536    1098.021
  Degrees of freedom                                28          28
  P-value                                        0.000       0.000
  Scaling correction factor                                  1.816

User 

#### Model Comparison

To assess which model provides a better fit (and therefore to assess which level of measurement invariance holds) we can use the `anova()` function.  

Lets first compare the **configural model** and the **metric model**.

In [30]:
print(ro.r("anova(fit_configural, fit_metric)"))


Scaled Chi-Squared Difference Test (method = “satorra.bentler.2001”)

lavaan->lavTestLRT():  
   lavaan NOTE: The “Chisq” column contains standard test statistics, not the 
   robust test that should be reported per model. A robust difference test is 
   a function of two standard (not robust) statistics.
               Df   AIC   BIC  Chisq Chisq diff Df diff Pr(>Chisq)  
fit_configural 15 14812 14940 301.69                                
fit_metric     18 14815 14930 310.76     7.2119       3    0.06544 .
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1



We see that the metric model fits worse than the configural model. This is no suprise as we include more restrictions in the metric model. However, the fit is not significantly worse (p = .0654) and the BIC even favors the metric model. From this we may conclude that metric invariance holds but there is a huge BUT (see below).

Next, lets compare the **metric model** and the **scale model**.

In [32]:
print(ro.r("anova(fit_metric, fit_scale)"))


Scaled Chi-Squared Difference Test (method = “satorra.bentler.2001”)

lavaan->lavTestLRT():  
   lavaan NOTE: The “Chisq” column contains standard test statistics, not the 
   robust test that should be reported per model. A robust difference test is 
   a function of two standard (not robust) statistics.
           Df   AIC   BIC  Chisq Chisq diff Df diff Pr(>Chisq)
fit_metric 18 14815 14930 310.76                              
fit_scale  21 14814 14916 315.75     4.9207       3     0.1777



From this comparison we can conclude that scale invariance holds as the scale model is not significantly worse than the metric model.  

Now to the **BUT**: You can see in the model output of the **configural model** that the fit is quite bad (e.g. RMSEA =  0.177 (> .08)). This is not too surprising since research has shown that there are two subdimensions of SDO (Ho et al., 2015):
anti-egalitarianism and dominance. According to the theory, the first two items are
supposed to load on the dominance dimension (SDO-D), whereas the remaining two
items load on anti-egalitarianism (SDO-E).  

Therefore, we should be cautious with interpreting the comparisons above and first make sure that we find a good fitting model to our data.